## 모듈 설치

In [14]:
!pip install openai
!pip install python-dotenv
!pip install pyperclip
!pip install langchain
!pip install langchain-community
!pip install langchain-openai
!pip install youtube-transcript-api
!pip install tiktoken
!pip install google-api-python-client
!pip install yt-dlp


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip insta

## 모듈 가져오기

In [21]:
# OpenAI API 사용
import os
from dotenv import load_dotenv
import pyperclip
import re
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import load_summarize_chain
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from googleapiclient.discovery import build

In [22]:
# 환경 변수 로드
load_dotenv()

True

## 영상 목록 DB로

#### csv 데이터 프레임으로 가져오기

In [ ]:
df_damoa = pd.read_csv('damoa_videos_deduplicated.csv', encoding='utf-8')

# 요약 콜럼 추가
df_damoa['요약'] = None

# 콜럼 순서 바꾸기
new_order = ['제목','기관','키워드', '카테고리목록','요약','링크']
df_damoa = df_damoa[new_order]

#### 키워드 추가

In [14]:
import pandas as pd
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage

# 파일 데이터 꼬임 방지를 위해 새 변수에 데이터 프레임 할당
df_damoa_kw = df_damoa

# 대답의 일관성을 위해 temperature 0으로 설정
keyword_model = init_chat_model(
                                model= "gpt-4o-mini",
                                temperature= 0
                                )

# gpt 추천 gpt 이용해서 답변 비교하는 프롬프트
def add_keyword(col1, col2):
    prompt = (
    "아래는 유튜브 영상의 제목과 카테고리 목록입니다.\n"
    "카테고리 목록은 여러 개의 항목이 세미콜론(;)으로 구분되어 있습니다.\n"
    "각 항목은 '카테고리명/세부주제' 형식입니다.\n"
    "다음 규칙에 따라 키워드를 선정하세요:\n"
    "- 카테고리 목록의 모든 카테고리명과 세부주제(예: 요리, 건강, 친구, 선거 등)는 각각 반드시 키워드에 모두 포함시키세요.\n"
    "- 중복된 카테고리명 또는 세부주제는 한 번만 포함시키면 됩니다.\n"
    "- 그 외에도, 영상 제목을 참고하여 추가적으로 중요한 키워드가 있다면 포함하세요.\n"
    "- 키워드는 단어 또는 짧은 구로 작성하고, 콤마(,)로 구분하여 한 줄로만 출력하세요.\n"
    "- 설명, 이유, 기타 문장은 절대 쓰지 마세요. 오로지 키워드만 출력하세요.\n\n"
    f"[영상 제목]\n{col1}\n\n"
    f"[영상 카테고리]\n{col2}\n\n"
)
    response = keyword_model.invoke([HumanMessage(content=prompt)]).content.strip()
    return response

keyword_added = []
for idx, row in df_damoa_kw.iterrows():
    result = add_keyword(row['제목'],row['카테고리목록'])
    keyword_added.append(result)

df_damoa_kw['키워드'] = keyword_added

In [23]:
df_damoa_kw.to_csv('data/damoavideokw.csv')

#### DB 구성하기

In [24]:
# 문제 1.1: SQLite3를 사용하여 다음의 조건을 만족하는 데이터베이스를 연결하고 테이블을 생성한다.

conn_moavdb = sqlite3.connect('data/moav.db')

# 테이블 생성 SQL 쿼리 변수 할당
# 코드 반복 실행시 편의를 위해 기존 생성 테이블 초기화 하고 시작
create1 = '''
DROP TABLE IF EXISTS Damoavideo;
CREATE TABLE Damoavideo(
순번 INTEGER PRIMARY KEY,
제목 TEXT, 
기관 TEXT,
키워드 TEXT,
카테고리목록 TEXT,
요약 TEXT,
링크 TEXT
);'''

# 문제 지시 테이블 생성
try:
    cursor_moavdb = conn_moavdb.cursor()
    cursor_moavdb.executescript(create1)
    conn_moavdb.commit()

except sqlite3.Error as sql3err1:
    print('SQLite ERROR:', sql3err1)

finally: # 항상 실행됨으로 db 닫아야 할 때 필수
    conn_moavdb.close()

In [25]:
def load_and_prepare_data(df_path):
    # 데이터 로드
    # 순번(인덱스)을 primary key로 사용할 수 있도록 변환
    # 각 행을 튜플로 변환 (순번,'제목','기관','키워드','카테고리목록','링크')
    df_name = pd.read_csv(df_path)
    
    data_tuples = [
        (
            int(idx),              
            str(row['제목']),         
            str(row['기관']),        
            str(row['키워드']),       
            str(row['카테고리목록']),
            str(row['요약']),        
            str(row['링크'])
        )
        for idx, row in df_name.iterrows()
    ]
    return data_tuples

def insert_data_to_db(db_path, table_name, data_tuples):
    insert_sql = f'''
    INSERT INTO {table_name}
    (순번,제목,기관,키워드,카테고리목록,요약,링크)
    VALUES (?,?,?,?,?,?,?)
    '''
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        # 데이터 중복 입력 방지
        cursor.execute(f'DELETE FROM {table_name};')
        cursor.executemany(insert_sql, data_tuples)
        conn.commit()


In [26]:
data = load_and_prepare_data('data/damoavideokw.csv')
insert_data_to_db('data/moav.db', 'Damoavideo', data)
print("데이터가 입력되었습니다.")


데이터가 입력되었습니다.


#### 입력 값을 받아 관련 영상을 추천하는 노드

In [28]:
from typing import TypedDict, Annotated, List, Dict, Optional

class State(TypedDict):
    video_summary: str
    recommendation: List[Dict[str, str]]

In [29]:
State['video_summary'] = '아이와 놀아주는 방법'

TypeError: '_TypedDictMeta' object does not support item assignment

In [23]:
import operator
import re
from langchain_core.runnables import Runnable
from langchain_community.utilities import SQLDatabase
from langchain.chat_models import init_chat_model

class VideoRecommendationNode:
    """
    영상 추천 파이프라인의 메인 클래스. LLM과 DB를 연결해 영상 추천 결과를 생성하고 필터링한다.
    """
    def __init__(self, runnable: Runnable, db: SQLDatabase, chunk_size: int = 50) -> None:
        self.__runnable = runnable
        self.__db = db
        self.__chunk_size = chunk_size

    def _build_prompt_header(self, user_input: str, header_note: str) -> List[str]:
        """
        LLM 프롬프트의 공통 헤더(사용자 관심사 + 안내문) 리스트를 반환
        user_input: 추천 기준(예: 영상 요약문 등)
        header_note: 프롬프트 중간 안내 문구
        """
        return [
            "Below is a description of the user's interest:",
            user_input,
            f"\n{header_note}"
        ]

    def __call__(self, state: State) -> State:
        """
        입력 요약을 받아 영상 추천 프롬프트를 만들고 LLM으로 결과를 추천받아 반환
        (추천 필터링까지 한 번에 수행)
        state: State 입력
        return: 추천 결과 포함된 VideoRecommendState
        """
        user_input = state.get("video_summary") or ""
        if not user_input:
            print("입력값 없음 (summary 필드 필요)")
            return state

        query = """
            SELECT 제목, 키워드, 요약, 링크
            FROM Damoavideo;
        """
        rows = self.__db._execute(query)
        if not isinstance(rows, list):
            print("데이터베이스 응답 오류")
            return state

        chunks = [rows[i:i+self.__chunk_size] for i in range(0, len(rows), self.__chunk_size)]
        row_map = {str(row.get('제목', '')).strip(): row for row in rows}
        all_recommended = {}

        for chunk in chunks:
            prompt_lines = self._build_prompt_header(user_input, "Here is a list of videos:")
            for row in chunk:
                prompt_lines.append(
                    f"Title: {row['제목']}\nKeywords: {row['키워드']}\nSummary: {row['요약']}"
                )
            prompt_lines.append(
                """
                Please recommend up to 3 videos that are most closely related to the user's input.
                Do not recommend videos that are not clearly relevant.
                Output in the same language as the user's input.
                Only include one video per line in the following format:
                Title: [video title]
                Link: [video link]
                Reason: [a short reason why this video is relevant to the user's input]
                Separate each video with a blank line.
                """
            )
            prompt = "\n\n".join(prompt_lines)
            try:
                response = self.__runnable.invoke(prompt)
            except Exception as e:
                print(f"LLM 응답 실패: {e}")
                continue

            if not hasattr(response, "content") or not isinstance(response.content, str):
                print("LLM 응답 형식 오류: content 속성이 없거나 문자열이 아님")
                continue

            blocks = response.content.strip().split('\n\n')
            for block in blocks:
                title_match = re.search(r"^Title:\s*(.+)$", block, re.MULTILINE)
                link_match = re.search(r"^Link:\s*(.+)$", block, re.MULTILINE)
                reason_match = re.search(r"^Reason:\s*(.+)$", block, re.MULTILINE)
                if title_match:
                    title = title_match.group(1).strip()
                    link = link_match.group(1).strip() if link_match else ""
                    reason = reason_match.group(1).strip() if reason_match else ""
                    all_recommended[title] = {"title": title, "link": link, "reason": reason}

        recommendations = []
        for title, data in all_recommended.items():
            row = row_map.get(title)
            if row:
                recommendations.append({
                    "title": title,
                    "link": row['링크'],
                    "reason": data['reason']
                })

        if recommendations:
            review_lines = self._build_prompt_header(user_input, "Here is a list of recommended videos:")
            for rec in recommendations:
                review_lines.append(f"Title: {rec['title']}\nLink: {rec['link']}\nReason: {rec['reason']}")
            review_lines.append(
                """
                From the recommended list above, filter and return only the videos that are most relevant to the user's input.
                Exclude any videos that are not clearly relevant.
                Output in the same language as the user's input.
                Only include one video per line in the following format:
                Title: [video title]
                Link: [video link]
                Reason: [a short reason why this video is relevant to the user's input]
                Separate each video with a blank line.
                """
            )
            review_prompt = "\n\n".join(review_lines)
            try:
                review_response = self.__runnable.invoke(review_prompt)
            except Exception as e:
                print(f"LLM 필터링 실패: {e}")
                return {"recommendation": recommendations, **state}

            if not hasattr(review_response, "content") or not isinstance(review_response.content, str):
                print("LLM 필터링 응답 형식 오류")
                return {"recommendation": recommendations, **state}

            filtered_titles = set()
            blocks = review_response.content.strip().split('\n\n')
            for block in blocks:
                title_match = re.search(r"^Title:\s*(.+)$", block, re.MULTILINE)
                if title_match:
                    title = title_match.group(1).strip()
                    filtered_titles.add(title)

            recommendations = [rec for rec in recommendations if rec['title'] in filtered_titles]

        return {"recommendation": recommendations}

In [27]:
recommend_model = init_chat_model(model='gpt-4o-mini')
db = SQLDatabase.from_uri("sqlite:///data/moav.db")
recommender = VideoRecommendationNode(recommend_model, db)

In [ ]:
input_state = state.get("video_summary")
output_state = recommender(.get)

recommendations = output_state["recommendation"]

입력값 없음 (summary 필드 필요)


KeyError: 'recommendation'

In [26]:
print(output_state['recommendation'])

KeyError: 'recommendation'

In [15]:
recommendations = [{'title': '지금 내 마음은 무슨 색일까 감정을 이해하고 표현하는 법', 'link': 'https://www.damoa.or.kr/main/cmf_video.php?no=168', 'reason': '이 영상은 감정을 이해하고 표현하는 방법을 다루고 있어, 발달장애 아동이 감정을 소통하는 데 도움이 되는 내용입니다.'}, {'title': '발달장애 및 지적장애 학생을 위한 감정이해하기 지도 프로그램', 'link': 'https://www.damoa.or.kr/main/cmf_video.php?no=2076', 'reason': '감정 이해 능력을 향상시키기 위한 프로그램을 소개하여, 발달장애 아동들이 자신의 감정을 표현하고 이해하는 데 필요한 정보를 제공합니다.'}, {'title': '친구들과 사이좋게 지내는 방법', 'link': 'https://www.damoa.or.kr/main/cmf_video.php?no=78', 'reason': '또래 관계를 맺는 방법에 대한 내용을 담고 있어, 발달장애 아동이 건강한 인간관계를 형성하는 데 도움이 될 수 있습니다.'}, {'title': '발달장애 청소년 친구들과 함께 쿠키 만들었어요! (with. 유튜버 치도)(청소년발달장애학생 방과후활동서비스)', 'link': 'https://www.damoa.or.kr/main/cmf_video.php?no=1614', 'reason': '이 영상은 발달장애 청소년들이 함께 요리 활동을 통해 소통하고 건강한 관계를 형성하는 모습을 보여주어, 또래와의 자연스러운 교류를 지원합니다.'}, {'title': '발달장애 아동과 청소년으로 구성된 레인보우 뮤지션팀을 소개합니다.', 'link': 'https://www.damoa.or.kr/main/cmf_video.php?no=2027', 'reason': '이 영상은 발달장애 아동과 청소년들이 예술 활동을 통해 자신을 표현하고, 공동체를 이루며 건강한 관계를 형성하는 모습을 보여줍니다.'}, {'title': '발달장애 예술 감독이 특별한 이유?', 'link': 'https://www.damoa.or.kr/main/cmf_video.php?no=3992', 'reason': '이 영상은 발달장애를 가진 아이들이 예술 활동을 통해 감정을 표현하는 중요성을 다루고 있어, 사용자의 관심사와 밀접하게 관련이 있습니다.'}, {'title': '발달장애 학생들의 하루 일과', 'link': 'https://www.damoa.or.kr/main/cmf_video.php?no=1663', 'reason': '이 영상은 발달장애 학생들이 어떻게 일상 생활에서 사회와 어우러지는지를 보여주며, 또래와의 관계 형성의 중요성을 강조합니다.'}]

#### 추천 영상 유튜브 링크 가져오기

In [16]:
import os
import subprocess
import sqlite3
import re
import requests
import time
import concurrent.futures
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from youtube_transcript_api import (
    YouTubeTranscriptApi,
    TranscriptsDisabled,
    NoTranscriptFound,
    VideoUnavailable,
    YouTubeRequestFailed
)
import math

db_path = 'data/moav.db'

def extract_youtube_links_from_page(url):
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/114.0.0.0 Safari/537.36"
        )
    }
    retries = 3
    delay = 2

    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            youtube_links = []
            for iframe in soup.find_all("iframe"):
                src = iframe.get("src")
                if src and "youtube.com" in src:
                    match = re.search(r"(?:/embed/|/v/|\?v=)([a-zA-Z0-9_-]{11})", src)
                    if match:
                        video_id = match.group(1)
                        youtube_links.append(f"https://www.youtube.com/watch?v={video_id}")
            return youtube_links if youtube_links else ["[❌] 유튜브 영상 링크를 찾을 수 없습니다."]
        except Exception as e:
            if attempt < retries - 1:
                time.sleep(delay * (2 ** attempt))
                continue
            return [f"[❌] 오류 발생: {e}"]

def get_youtube_transcript(video_url, languages=['ko', 'en'], preserve_formatting=False):
    try:
        if "v=" in video_url:
            video_id = video_url.split("v=")[-1].split("&")[0]
        else:
            video_id = video_url.strip().split("/")[-1]
        transcript = YouTubeTranscriptApi().get_transcript(
            video_id,
            languages=languages
        )
        return " ".join([snippet['text'] for snippet in transcript])
    except (TranscriptsDisabled, NoTranscriptFound, VideoUnavailable, YouTubeRequestFailed):
        return None
    except Exception as e:
        return None

def summarize_text_llm(text, model, prompt_template=None):
    if prompt_template is None:
        prompt_template = "다음 유튜브 영상 자막을 한국어로 간결하게 요약해줘:\n\n{text}"
    prompt = prompt_template.format(text=text)
    try:
        return model.invoke(prompt).content
    except Exception:
        return None

def is_summary_exist(db_path, page_url):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute("SELECT 요약 FROM Damoavideo WHERE 링크 = ?", (page_url,))
    result = cur.fetchone()
    cur.close()
    conn.close()
    if result is None:
        return False
    summary = result[0]
    if summary is None:
        return False
    if isinstance(summary, float) and math.isnan(summary):
        return False
    # 'nan'(문자열)도 결측치 취급
    if str(summary).strip().lower() == 'nan':
        return False
    return True

def save_summary_to_db(db_path, page_url, summary):
    if is_summary_exist(db_path, page_url):
        return  # 이미 값이 있으면 skip
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute("UPDATE Damoavideo SET 요약 = ? WHERE 링크 = ?", (summary, page_url))
    conn.commit()
    cur.close()
    conn.close()

def remove_video_from_db(db_path, page_url):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute("DELETE FROM Damoavideo WHERE 링크 = ?", (page_url,))
    conn.commit()
    cur.close()
    conn.close()

def process_video(rec, model):
    results = []
    page_url = rec.get("link")
    youtube_links = extract_youtube_links_from_page(page_url)
    for y_url in youtube_links:
        transcript = get_youtube_transcript(y_url)
        if transcript:
            summary = summarize_text_llm(transcript, model)
            if summary:
                save_summary_to_db(db_path, page_url, summary)
            results.append({
                "youtube_url": y_url,
                "transcript": transcript,
                "summary": summary
            })
        else:
            remove_video_from_db(db_path, page_url)
    return {**rec, "summaries": results} if results else None

def process_recommendations_with_youtube(recommendations, model):
    def process_one(rec):
        page_url = rec.get("link")
        youtube_links = extract_youtube_links_from_page(page_url)
        summaries = []
        valid = False
        for y_url in youtube_links:
            transcript = get_youtube_transcript(y_url)
            if transcript:
                valid = True
                summary = summarize_text_llm(transcript, model)
                if summary:
                    save_summary_to_db(db_path, page_url, summary)
                summaries.append({
                    "youtube_url": y_url,
                    "transcript": transcript,
                    "summary": summary
                })
            else:
                remove_video_from_db(db_path, page_url)
        if valid:
            return {**rec, "youtube_links": youtube_links, "summaries": summaries}
        else:
            return None
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(process_one, recommendations))
    return [r for r in results if r is not None]

In [17]:
extract_youtube_links_from_page('https://www.damoa.or.kr/main/cmf_video.php?no=168')

['https://www.youtube.com/watch?v=dK-73HM8qv4']

In [18]:
result = process_recommendations_with_youtube(recommendations, recommend_model)

In [19]:
result

[{'title': '지금 내 마음은 무슨 색일까 감정을 이해하고 표현하는 법',
  'link': 'https://www.damoa.or.kr/main/cmf_video.php?no=168',
  'reason': '이 영상은 감정을 이해하고 표현하는 방법을 다루고 있어, 발달장애 아동이 감정을 소통하는 데 도움이 되는 내용입니다.',
  'youtube_links': ['https://www.youtube.com/watch?v=dK-73HM8qv4'],
  'summaries': [{'youtube_url': 'https://www.youtube.com/watch?v=dK-73HM8qv4',
    'transcript': 'oo 아 으 박 넣어보세요 [음악] 으 허 아 네 [음악] 홍대 교총 뭐 열어봐도 다른거 [음악] 픽 for 화질 제프 합당 인간을 초암 색 일단은 노랑 색 꼭 후 은색 때 이 썸 먹어봐도 됐죠 [음악] 아 중 짧다 아 으 으 으 na [음악] 5 어서와 여긴 b 마음속 감정 친구들을 만나서 이야기를 나눠 봐 오 두고 외고 오나 또 누구야 멀고 있어 [음악] 나는 슬픔이라고 해 5 어 애써 쌓은 블록이 망가졌을 때 엄마가 속상할 때 5 친구랑 더 놀고 싶은데 헤어져야 할 때 주는 그런 마음 아 이러면 슬픔의 고나 괴롭고 아픈 마음이 술 뿐이야 어 그런걸 슬프고 굴어 5 잠깐만 선물이 일쑤 이거 다 주는 거야 으 we 으 아 으 으 아 짜증나 괭 와나 웃은 내리 써 나는 화나 늘 기분이 나쁘지 그 나는 짜증이 다 못 마땅해 으 동생이 드림을 망쳐 났을 때 음 으 서 있는데 친구가 끼어들 때 엄마가 재밌는 경상을 그만 보라고 할 때 뜨는 나음 마음에 들지 않아 성질이 나는 게 화아 으 못마땅해서 기분 나쁜 게 짜증이 야 음 멎어 그럴 때 흔하고 짠 맛 2 그럴땐 표현이 하는게 맞는 거야 그런건 나쁜게 아니야 올 싸 맞아 맞아 택배 때 자 한것 봐 5 [음악] e58 풀어놓는 새 [음악] 건 또 무슨 냄새야 조심하라고 [음악] 아니냔 안녕 

#### 토큰 계산기 추가 버전

In [32]:
import os
import subprocess
import sqlite3
import re
import requests
import time
import random
import concurrent.futures
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from youtube_transcript_api import (
    YouTubeTranscriptApi,
    TranscriptsDisabled,
    NoTranscriptFound,
    VideoUnavailable,
    YouTubeRequestFailed
)
import math
import tiktoken

db_path = 'data/moav.db'


def count_tokens(text, model_name="gpt-4o"):
    try:
        encoding = tiktoken.encoding_for_model(model_name)
    except Exception:
        encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(text))


def extract_youtube_links_from_page(url):
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/114.0.0.0 Safari/537.36"
        )
    }
    retries = 3
    delay = 2
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=10)
            soup = BeautifulSoup(response.text, 'html.parser')
            youtube_links = []
            for iframe in soup.find_all("iframe"):
                src = iframe.get("src")
                if src and "youtube.com" in src:
                    match = re.search(r"(?:/embed/|/v/|\?v=)([a-zA-Z0-9_-]{11})", src)
                    if match:
                        video_id = match.group(1)
                        youtube_links.append(f"https://www.youtube.com/watch?v={video_id}")
            return youtube_links if youtube_links else []
        except Exception as e:
            if attempt < retries - 1:
                time.sleep(delay * (2 ** attempt))
                continue
            return []

def get_youtube_transcript(video_url, languages=['ko', 'en'], preserve_formatting=False):
    try:
        if "v=" in video_url:
            video_id = video_url.split("v=")[-1].split("&")[0]
        else:
            video_id = video_url.strip().split("/")[-1]
        transcript = YouTubeTranscriptApi().get_transcript(
            video_id,
            languages=languages
        )
        return " ".join([snippet['text'] for snippet in transcript])
    except (TranscriptsDisabled, NoTranscriptFound, VideoUnavailable, YouTubeRequestFailed):
        return None
    except Exception as e:
        return None

def summarize_text_llm(text, model, prompt_template=None, model_name='gpt-4o'):
    if prompt_template is None:
        prompt_template = "다음 유튜브 영상 자막을 한국어로 간결하게 요약해줘:\n\n{text}"
    prompt = prompt_template.format(text=text)
    prompt_tokens = count_tokens(prompt, model_name)
    try:
        result = model.invoke(prompt)
        content = getattr(result, 'content', None) or str(result)
        completion_tokens = count_tokens(content, model_name)
        return content, prompt_tokens, completion_tokens
    except Exception:
        return None, prompt_tokens, 0

def is_summary_exist(db_path, page_url):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute("SELECT 요약 FROM Damoavideo WHERE 링크 = ?", (page_url,))
    result = cur.fetchone()
    cur.close()
    conn.close()
    if result is None:
        return False
    summary = result[0]
    if summary is None:
        return False
    if isinstance(summary, float) and math.isnan(summary):
        return False
    if str(summary).strip().lower() == 'nan':
        return False
    return True

def save_summary_to_db(db_path, page_url, summary):
    if is_summary_exist(db_path, page_url):
        return
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute("UPDATE Damoavideo SET 요약 = ? WHERE 링크 = ?", (summary, page_url))
    conn.commit()
    cur.close()
    conn.close()

def remove_video_from_db(db_path, page_url):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute("DELETE FROM Damoavideo WHERE 링크 = ?", (page_url,))
    conn.commit()
    cur.close()
    conn.close()

def process_video(rec, model, model_name='gpt-4o'):
    results = []
    page_url = rec.get("link")
    youtube_links = extract_youtube_links_from_page(page_url)
    total_prompt_tokens, total_completion_tokens = 0, 0
    for y_url in youtube_links:
        time.sleep(random.uniform(0.5, 2.0))
        transcript = get_youtube_transcript(y_url)
        if transcript:
            summary, ptok, ctok = summarize_text_llm(transcript, model, model_name=model_name)
            total_prompt_tokens += ptok
            total_completion_tokens += ctok
            if summary:
                save_summary_to_db(db_path, page_url, summary)
            results.append({
                "youtube_url": y_url,
                "transcript": transcript,
                "summary": summary,
                "prompt_tokens": ptok,
                "completion_tokens": ctok
            })
        else:
            remove_video_from_db(db_path, page_url)
    return {**rec, "summaries": results, "prompt_tokens": total_prompt_tokens, "completion_tokens": total_completion_tokens} if results else None

def process_recommendations_with_youtube(recommendations, model, model_name='gpt-4o'):
    def process_one(rec):
        page_url = rec.get("link")
        youtube_links = extract_youtube_links_from_page(page_url)
        summaries = []
        valid = False
        total_prompt_tokens, total_completion_tokens = 0, 0
        for y_url in youtube_links:
            time.sleep(random.uniform(0.5, 2.0))
            transcript = get_youtube_transcript(y_url)
            if transcript:
                valid = True
                summary, ptok, ctok = summarize_text_llm(transcript, model, model_name=model_name)
                total_prompt_tokens += ptok
                total_completion_tokens += ctok
                if summary:
                    save_summary_to_db(db_path, page_url, summary)
                summaries.append({
                    "youtube_url": y_url,
                    "transcript": transcript,
                    "summary": summary,
                    "prompt_tokens": ptok,
                    "completion_tokens": ctok
                })
            else:
                remove_video_from_db(db_path, page_url)
        if valid:
            return {**rec, "youtube_links": youtube_links, "summaries": summaries, "prompt_tokens": total_prompt_tokens, "completion_tokens": total_completion_tokens}
        else:
            return None
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(process_one, recommendations))
    total_prompt_tokens = sum(r["prompt_tokens"] for r in results if r)
    total_completion_tokens = sum(r["completion_tokens"] for r in results if r)
    return [r for r in results if r is not None], total_prompt_tokens, total_completion_tokens


In [33]:
result = process_recommendations_with_youtube(recommendations, recommend_model)

In [34]:
result

([{'title': '지금 내 마음은 무슨 색일까 감정을 이해하고 표현하는 법',
   'link': 'https://www.damoa.or.kr/main/cmf_video.php?no=168',
   'reason': '이 영상은 감정을 이해하고 표현하는 방법을 다루고 있어, 발달장애 아동이 감정을 소통하는 데 도움이 되는 내용입니다.',
   'youtube_links': ['https://www.youtube.com/watch?v=dK-73HM8qv4'],
   'summaries': [{'youtube_url': 'https://www.youtube.com/watch?v=dK-73HM8qv4',
     'transcript': 'oo 아 으 박 넣어보세요 [음악] 으 허 아 네 [음악] 홍대 교총 뭐 열어봐도 다른거 [음악] 픽 for 화질 제프 합당 인간을 초암 색 일단은 노랑 색 꼭 후 은색 때 이 썸 먹어봐도 됐죠 [음악] 아 중 짧다 아 으 으 으 na [음악] 5 어서와 여긴 b 마음속 감정 친구들을 만나서 이야기를 나눠 봐 오 두고 외고 오나 또 누구야 멀고 있어 [음악] 나는 슬픔이라고 해 5 어 애써 쌓은 블록이 망가졌을 때 엄마가 속상할 때 5 친구랑 더 놀고 싶은데 헤어져야 할 때 주는 그런 마음 아 이러면 슬픔의 고나 괴롭고 아픈 마음이 술 뿐이야 어 그런걸 슬프고 굴어 5 잠깐만 선물이 일쑤 이거 다 주는 거야 으 we 으 아 으 으 아 짜증나 괭 와나 웃은 내리 써 나는 화나 늘 기분이 나쁘지 그 나는 짜증이 다 못 마땅해 으 동생이 드림을 망쳐 났을 때 음 으 서 있는데 친구가 끼어들 때 엄마가 재밌는 경상을 그만 보라고 할 때 뜨는 나음 마음에 들지 않아 성질이 나는 게 화아 으 못마땅해서 기분 나쁜 게 짜증이 야 음 멎어 그럴 때 흔하고 짠 맛 2 그럴땐 표현이 하는게 맞는 거야 그런건 나쁜게 아니야 올 싸 맞아 맞아 택배 때 자 한것 봐 5 [음악] e58 풀어놓는 새 [음악] 건 또 무슨 냄새야 조심하라고 [음악] 아

## 합치기 용

In [ ]:
import operator
import re
from langchain_core.runnables import Runnable
from langchain_community.utilities import SQLDatabase
from langchain.chat_models import init_chat_model

class VideoRecommendationNode:
    """
    영상 추천 파이프라인의 메인 클래스. LLM과 DB를 연결해 영상 추천 결과를 생성하고 필터링한다.
    """
    def __init__(self, runnable: Runnable, chunk_size: int = 50) -> None:
        self.__runnable = runnable
        self.__chunk_size = chunk_size

    def _build_prompt_header(self, user_input: str, header_note: str) -> List[str]:
        """
        LLM 프롬프트의 공통 헤더(사용자 관심사 + 안내문) 리스트를 반환
        user_input: 추천 기준(예: 영상 요약문 등)
        header_note: 프롬프트 중간 안내 문구
        """
        return [
            "Below is a description of the user's interest:",
            user_input,
            f"\n{header_note}"
        ]

    def __call__(self, state: State) -> State:
        """
        입력 요약을 받아 영상 추천 프롬프트를 만들고 LLM으로 결과를 추천받아 반환
        (추천 필터링까지 한 번에 수행)
        state: State 입력
        return: 추천 결과 포함된 VideoRecommendState
        """
        node_name = '--- Video Recommendation Node'
        print(f'\n{node_name} {'-' * (79 - len('\n') - len(node_name) - 1)}')  

        user_input = state.get("video_summary") or ""
        if not user_input:
            print("입력값 없음 (summary 필드 필요)")
            return state

        query = """
            SELECT 제목, 키워드, 요약, 링크
            FROM Damoavideo;
        """
        db = SQLDatabase.from_uri("sqlite:///data/moav.db")
        rows = db._execute(query)
        if not isinstance(rows, list):
            print("데이터베이스 응답 오류")
            return state

        chunks = [rows[i:i+self.__chunk_size] for i in range(0, len(rows), self.__chunk_size)]
        row_map = {str(row.get('제목', '')).strip(): row for row in rows}
        all_recommended = {}

        for chunk in chunks:
            prompt_lines = self._build_prompt_header(user_input, "Here is a list of videos:")
            for row in chunk:
                prompt_lines.append(
                    f"Title: {row['제목']}\nKeywords: {row['키워드']}\nSummary: {row['요약']}"
                )
            prompt_lines.append(
                """
                Please recommend up to 3 videos that are most closely related to the user's input.
                Do not recommend videos that are not clearly relevant.
                Output in the same language as the user's input.
                Only include one video per line in the following format:
                Title: [video title]
                Link: [video link]
                Reason: [a short reason why this video is relevant to the user's input]
                Separate each video with a blank line.
                """
            )
            prompt = "\n\n".join(prompt_lines)
            try:
                response = self.__runnable.invoke(prompt)
            except Exception as e:
                print(f"LLM 응답 실패: {e}")
                continue

            if not hasattr(response, "content") or not isinstance(response.content, str):
                print("LLM 응답 형식 오류: content 속성이 없거나 문자열이 아님")
                continue

            blocks = response.content.strip().split('\n\n')
            for block in blocks:
                title_match = re.search(r"^Title:\s*(.+)$", block, re.MULTILINE)
                link_match = re.search(r"^Link:\s*(.+)$", block, re.MULTILINE)
                reason_match = re.search(r"^Reason:\s*(.+)$", block, re.MULTILINE)
                if title_match:
                    title = title_match.group(1).strip()
                    link = link_match.group(1).strip() if link_match else ""
                    reason = reason_match.group(1).strip() if reason_match else ""
                    all_recommended[title] = {"title": title, "link": link, "reason": reason}

        recommendations = []
        for title, data in all_recommended.items():
            row = row_map.get(title)
            if row:
                recommendations.append({
                    "title": title,
                    "link": row['링크'],
                    "reason": data['reason']
                })

        if recommendations:
            review_lines = self._build_prompt_header(user_input, "Here is a list of recommended videos:")
            for rec in recommendations:
                review_lines.append(f"Title: {rec['title']}\nLink: {rec['link']}\nReason: {rec['reason']}")
            review_lines.append(
                """
                From the recommended list above, filter and return only the videos that are most relevant to the user's input.
                Exclude any videos that are not clearly relevant.
                Output in the same language as the user's input.
                Only include one video per line in the following format:
                Title: [video title]
                Link: [video link]
                Reason: [a short reason why this video is relevant to the user's input]
                Separate each video with a blank line.
                """
            )
            review_prompt = "\n\n".join(review_lines)
            try:
                review_response = self.__runnable.invoke(review_prompt)
            except Exception as e:
                print(f"LLM 필터링 실패: {e}")
                return {"recommendation": recommendations, **state}

            if not hasattr(review_response, "content") or not isinstance(review_response.content, str):
                print("LLM 필터링 응답 형식 오류")
                return {"recommendation": recommendations, **state}

            filtered_titles = set()
            blocks = review_response.content.strip().split('\n\n')
            for block in blocks:
                title_match = re.search(r"^Title:\s*(.+)$", block, re.MULTILINE)
                if title_match:
                    title = title_match.group(1).strip()
                    filtered_titles.add(title)

            recommendations = [rec for rec in recommendations if rec['title'] in filtered_titles]

        return {"recommendation": recommendations}